In [3]:
#imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

# Constants (All in SI units) and Data

Relevant physical constants

In [ ]:
class constants:
    """
    Parameters:
        k_B: Boltzmann constant in J/K
        hbar: reduced Planck's constant
        h: Planck constant
        amu: 1 amu in kg
        c: speed of light in m/s
        mu_B: Bohr magneton in J/T
        a_0: Bohr radius in m
        q_e: electron charge
        mu_0: vacuum permeability
        epsilon_0: vacuum permittivity
    """
    k_B = 1.380649e-23  # Boltzmann constant in J/K
    hbar = 1.054572e-34  # reduced Planck's constant
    h = hbar * 2 * np.pi  # Planck constant
    amu = 1.660539066e-27  # 1 amu in kg
    c = 299792458  # speed of light in m/s
    mu_B = 9.2740100657e-24  # Bohr magneton in J/T
    a_0 = 5.29177210544e-11  # Bohr radius in m
    q_e = 1.602176634-19  # electron charge
    mu_0 = 1.25663706127e-6  # vacuum permeability
    epsilon_0 = 8.8541878188e-12  # vacuum permittivity

imported data as numpy arrays

In [26]:
CsVsBPath = "/Users/mouhamedmbengue/Desktop/code/chinLabCode/LiCs-CodebaseV2/data/Cs_a_vs_B.txt"
LiCs_aVsBPath = "/Users/mouhamedmbengue/Desktop/code/chinLabCode/LiCs-CodebaseV2/data/LiCs_a_vs_B.txt"

CsVsBData2 = np.loadtxt(fname=CsVsBPath, delimiter=",", skiprows=1)
LiCs_aVsBData = np.loadtxt(fname=LiCs_aVsBPath, delimiter=",", skiprows=1)

# Atomic Properties

Class for storing properties of Cesium 133

In [ ]:
class Cs133:
    """This class stores atomic properties of cesium-133."""
    mass = 132.905451931 * constants.amu  # atomic mass in kg
    I = 7/2  # nuclear spin
    g_I = -0.00039885395  # nuclear g-factor

    class GS: #ground state
        g_j = 2.002540261 #fine structure landé g-factor
        aHF = constants.h * 2.2981579425e9 #hyperfine structure constant in J

    class D1: #D1 transition
        omega = 2 * np.pi * 335.116048807e12  # angular frequency in Hz
        Gamma = 2 * np.pi * 4.575e6  # decay rate in Hz
        wavelength = 894.59295987e-9  # wavelength (in vaccum) in m
        g_j = 0.665900 #fine structure landé g-factor
        aHF = constants.h * 291.9201e6 #hyperfine structure constant in J

    class D2: #D2 transition
        omega = 2 * np.pi * 351.72571850e12  # angular frequency in Hz
        Gamma = 2 * np.pi * 5.234e6  # decay rate in Hz
        wavelength = 852.34727582e-9  # wavelength (in vacuum) in m
        g_j = 1.33408749 #fine structure landé g-factor
        aHF = constants.h * 50.28827e6 #hyperfine structure constant in J

Class for storing properties of Lithium 6

In [ ]:
class Li6:
    """This class stores atomic properties of lithium-6."""
    mass = 6.0151214 * constants.amu  # atomic mass in kg
    I = 1  # nuclear spin
    g_I = -0.000447654  # nuclear g-factor

    class GS: #ground state
        g_j = 2.0023010 #Total electronic g-factor 
        aHF = constants.h * 152.1368407e6 # hyperfine structure constrant in J

    class D1: #D1 transition
        omega = 2 * np.pi * 446.789634e12 # angular frequency in Hz
        Gamma = 2 * np.pi * 5.8724e6 # decay rate in Hz
        wavelength = 670.992421e-9 # wavelength in m
        g_j = 0.6668  #Total electronic g-factor
        aHF = constants.h * 17.386e6 # hyperfine structure constant in J
    
    class D2: #D2 transition
        omega = 2 * np.pi * 446.799677e12  # angular frequency in Hz
        Gamma = 2 * np.pi * 5.8724e6 # decay rate in Hz
        wavelength = 670.9977338e-9  # wavelength (in vacuum) in m
        g_j = 1.335 #Total electronic g-factor
        aHF = constants.h * -1.155e6 # hyperfine structure constant in J

# Calculations

## Breit-Rabi Energy

The Breit-rabi energy is computed using the following formula:

$$ E_{\ket{J=1/2, m_J, I, m_I}}  =  - \frac{\Delta E_{hfs}}{2(2I +1)} + g_i \mu_B m B \pm  \frac{\Delta E_{hfs}}{2} (1 + \frac{4mx}{2I + 1} + x^2)^{1/2}$$

taken from:

https://steck.us/alkalidata/cesiumnumbers.pdf

where:

$$ \Delta E_{hfs} = A_{hfs}(I + 1/2) $$

is the hyper fine splitting energy, and:

$$ x = \frac{(g_J - g_I)\mu_B B}{\Delta E_{hfs}} $$

For the purposes of this function:

$ coef1 = \frac{\Delta E_{hfs}}{2(2I +1)} $

$ coef2 = g_I \mu_B m B$

$coef3 = \frac{\Delta E_{hfs}}{2} (1 + \frac{4mx}{2I + 1} + x^2)^{1/2}$

In [112]:
def breitRabi(B, m, pm:str, atomType:int, freq:bool):
    """
    Function for computing the breit-rabi energy for Li6 or Cs133

    Parameters:
        B: Magnetic field in gauss
        m: Total magnetic quantum number
        pm: "p" or "m" decides between plus or minus in the formula
        atomType: atomic species, 6 for lithium or 133 for cesium
        freq: parameter to decide whether or not to return the answer as a frequency
    
    Returns:
        eBR: breit-rabi energy in joules or Hz depending on the freq parameter
    """

    #convert the field value to Tesla
    B = B * 1e-4

    #define relevant constants depending on atom type
    if atomType == 6: #Lithium
        g_I = Li6.g_I
        g_J = Li6.GS.g_j
        a_HF = Li6.GS.aHF
        I = Li6.I
    
    elif atomType == 133: #Cesium
        g_I = Cs133.g_I
        g_J = Cs133.GS.g_j
        a_HF = Cs133.GS.aHF 
        I = Cs133.I
    
    else:
        return "Error: atomType must be 6 or 133"
    

    #determine whether we are in the plus state or minus state
    if pm == "p":
        sign = 1
    elif pm == "m":
        sign = -1
    else:
        return "Error: pm must be p or m"
    
    #compute the hyperfine splitting energy
    E_hfs = a_HF * (I + 1/2)

    #compute the x-factor
    x = ((g_J - g_I) * constants.mu_B * B)/E_hfs

    # compute the three coefficients
    coef1 = E_hfs/( 2*(2*I + 1) )

    coef2 = g_I * constants.mu_B * m * B

    coef3 = E_hfs/2 * ( 1 + (4*m*x)/(2*I +1) + x**2)**(1/2)

    #compute breit-rabi energy
    E_br =  -coef1 + coef2 + sign*coef3

    #determine whether to return the answer as a frequency (Hz) or energy (J)
    if freq:
        return E_br / constants.h
    else:
        return E_br

breitRabi(900, 4, "p", 133, freq=True) - breitRabi(900, 3, "m", 133, freq=True)

11462235254.012493

## Inverse Breit-Rabi

It just implements root finding to determine the magnetic field from the breit rabi frequency

In [115]:
def bFromFreq(freq):
    """
    Function for determining the magnetic field needed to obtain a specific breit-rabi frequency split

    Parameters:
        freq: the frequency in Herz
    
    Returns:
        fieldMag: the magnetic field in Gauss
    """
    
    #define the function
    function = lambda BVal: breitRabi(BVal, 4, "p", 133, freq=True) - breitRabi(BVal, 3, "m", 133, freq=True) - freq

    #implement root finding and return the field value
    fieldMag = sp.optimize.fsolve(func=function, x0=892)

    return fieldMag[0]

bFromFreq(11.462e9)

np.float64(899.9087698425749)

## Scattering length calculations

Function for returning the scattering length of two Cs atoms in the |3, 3> state.

The data is from:
    
    Berninger et. al. "Feshbach resonances, weakly bound molecular states, and coupled-channel potentials for cesium at high magnetic fields" Phys. Rev. A 87, 032517 (2013)

It is stored in Cs_a_vs_B.txt.

In [22]:
def aCsCs(B:float):
    """
    aCsCs - returns the scattering length of two Cs atoms in the |3, 3> state.

    Parameters:
        B: The magnetic field strength in Gauss.
    
    Returns:
        a: The scattering length in Bohr radii.
    
    """
    #get the magnetic field strength and scatering length
    BCsData = CsVsBData2[:, 0]
    Cs_a = CsVsBData2[:, 1]

    #interpolate the data using a cubic spline
    spline = sp.interpolate.CubicSpline(BCsData, Cs_a, extrapolate=False)

    #return the scattering length value for a given field strength
    return spline(B)

Function for calculating the scattering length of two Li atoms of various species for a given magnetic field. 

Possible species inputs are "ab", "ac", or "bc" which represent the |1, 2>, |1, 3>, and |2, 3> states respectively.

Values are from table II of:

    Precise Characterization of $^{6}\mathrm{Li}$ Feshbach Resonances Using Trap-Sideband-Resolved RF Spectroscopy of Weakly Bound Molecules" by Zurn, et. al. PRL 110 13 135301 (2013)


In [122]:
def aLiLi(B, species):
    """
    aLiLi - returns the Li Li scattering lengths at field B between the species given in species ('ab', 'ac', or 'bc'). 

    Parameters:
        B: The magnetic field strength in Gauss.

        species: The state of the lithium atoms 'ab', 'ac', or 'bc'
    
    Returns:
        a: The scattering length in Bohr radii.
    """

    if species == "ab":
        abg = -1582
        delta = -262.3
        B0 = 832.18
    elif species == "ac":
        abg = -1770
        delta = -166.6
        B0 = 689.68
    elif species == "bc":
        abg = -1490
        delta = -222.3
        B0 = 809.76
    else:
        return "Error: Invalid species argument, must be ab, ac, or bc"
    
    return abg * (1 - delta / (B - B0))

Function for calculating the scattering length of one Li atom and one Cs atom given the magnetic field strength for various species.

The possible species are "a" or "b" which represent the |Li:ground>-|Cs:Ground> and |Li:Excited>-|Cs:Ground> states respectively

The constants are pulled from tables 3.2 and 3.3 of:

    FESHBACH AND EFIMOV RESONANCES IN A $^{6}\mathrm{Li}-^{133}\mathrm{Cs}$ ATOMIC MIXTURE by Jacob Johansen

In [125]:
def aLiCs(B, species):
    """
    aLiCs - returns the Li-Cs scattering length at a given magnetic field B, for Li atoms in the state given by species. 

    Parameters:
        B: The magnetic field in Gauss.
        species: The state of the lithium atom, "a" for ground, and "b" for excited
    
    Returns:
        a: The scattering length in Bohr radii.
    """
    
    if species == 'a':
        return -29.4 * (1 - (-58.21)/(B - 842.829) - (-4.55)/(B - 892.648))
    
    elif species == 'b':
        return -29.6 * (1 - (-0.37)/(B - 816.113) - (-57.45)/(B - 888.577) - (-4.22)/(B - 943.033))
    
    else:
        return "Error: Invalid species argument, must be ab, ac, or bc"

In [6]:
def aLiCs_ci(abf, fieldErr, invFlag:bool):
    """
    Calculate the confidence interval on the LiCs scattering length abf given the field error fielderr.
    If invflag is true, then abf and sabf will be interpreted as 1/abf and 1/sabf.
    """

    pass

Function for calculating the scattering length of different LiCs moleculat species by spline interpolation.

The data is taken from LiCs_a_vs_B.txt.

Possible options are "a", "b", or "c" which represent $LiCs$, $Li_2Cs$, and $Li_3Cs$ respectively.



In [21]:
def aLiCs_molscat(B, species):
    """
    aLiCs_molscat - returns scattering length a from molscat calculations stored in file LiCs_a_cs_B.txt

    Parameters:
        B: The magnetic field in gauss
        species: The LiCs molecular species, "a" for LiCs, "b" for Li2Cs, or "c" for Li3Cs

    Returns:
        a: The scattering length of the chosen molular species in Bohr radii
    """

    BVals = LiCs_aVsBData[:, 0]
    
    if species == "a":
        #get the LiCs scattering values
        aVals = LiCs_aVsBData[:, 1]
    
    elif species == "b":
        #get the Li2Cs scattering values
        aVals = LiCs_aVsBData[:, 2]
    
    elif species == "c":
        #get the Li3Cs scattering values
        aVals = LiCs_aVsBData[:, 3]
    
    else:
        return "Error: Invalid species, must be a, b, or c"

    spline = sp.interpolate.CubicSpline(BVals, aVals, extrapolate=False)

    return spline(B)

## Chemical Potentials

### BEC Chemical Potential

Class for calculating various quantities related to a Cs133 BEC trapped in a harmonic potential, the formulas are taken from:

https://arxiv.org/pdf/cond-mat/9806038.pdf (Pgs 3, 6, 7, 15, 14, 19, 28, and 31)

https://ultracold.uchicago.edu/sites/default/files/thesis/KrutikPatel_thesis.pdf (Pg 21)


In [170]:
class BEC_calc():
    """
    Class for computing various quantities related to a Cs133 BEC

    Parameters:
        fx: frequency in the x direction (Hz)
        fy: frequency in the y direction (Hz)
        fz: frequency in the z direction (Hz)
        N: the number of atoms?
        a: scattering length (bohr radii)
    
    Returns:
        a_ho: Harmonic Oscillator length (microns)
        muT: chemical potential (nK) #convert to temp by dividing by k_b
        pD: peakDensity
        R_tf: Thomas-Fermi Radii: (Rx, Ry, Rz) (meters)
        xi: Central healing Length (microns)
        TCrit: critical temperature (nK)
        TCor: correction to critical temperature (fraction) caused by finite size and non uniformity
        aCrit: critical scattering length for a spherical potential (m)
        vSound: sound speed (mm/s)
        lambdaDB: Wrong? deBroglie wavelength (microns)
        phaseDensity: Wrong? phase space density # find the formula
    """
    def __init__(self,fx, fy, fz, N, a, printRes:bool):
        self.fx=fx
        self.fy=fy
        self.fz=fz
        self.N=N
        self.a=a

        #create a dictionary to store all of the rounded results, if you use the functions, you'll get the exact result
        self.results = dict()

        #convert scattering length to meters
        self.aBr = a *constants.a_0

        #convert frequencies to angluar frequencies
        self.wx = 2*np.pi*fx
        self.wy = 2*np.pi*fy
        self.wz = 2*np.pi*fz

        #compute intermediate values such as the cesium interaction strength, average frequency, and geometric average frequencies
        self.gInt = 4*np.pi*(constants.hbar**2)*self.aBr / Cs133.mass

        self.w_ho = (self.wx*self.wy*self.wz)**(1/3)

        self.wAvg = np.mean([self.wx, self.wy, self.wz])

        #run all of the calculation functions
        self.a_hoCalc(printVal=False)
        self.muTCalc(printVal=False)
        self.pDCalc(printVal=False)
        self.tfCalc(printVal=False)
        self.xiCalc(printVal=False)
        self.ctCalc(printVal=False)
        self.aCritCalc(printVal=False)
        self.vCalc(printVal=False)
        self.lambdaCalc(printVal=False)
        self.phaseDensityCalc(printVal=False)

        if printRes:
            self.genRes()

    def a_hoCalc(self, printVal:bool):
        "function for computing the harmonic oscillator length, returns value in microns"
        self.a_ho = np.sqrt(constants.hbar/(Cs133.mass*self.w_ho))
        self.results["Harmonic Oscillator Length (microns)"] = round(self.a_ho * 1e6, 5)

        if printVal:
            return self.a_ho * 1e6

    def muTCalc(self, printVal:bool):
        "function for computing the chemical potential, returns value in nK"
        self.mu = (constants.hbar*self.w_ho/2)*(15*self.N*self.aBr/self.a_ho)**(2/5)
        self.muT = self.mu / constants.k_B * 1e9 #chemical potential in nK
        self.results["chemical Potential (nK)"] = round(self.muT, 5)

        if printVal:
            return self.muT

    def pDCalc(self, printVal:bool):
        "function for computing peak density"
        self.pD = self.mu/self.gInt
        self.results["Peak Density"] = round(self.pD, 5)

        if printVal:
            return self.pD

    def tfCalc(self, printVal:bool):
        "function for computing the thomas-fermi radius, returns the result in meters"
        rx = np.sqrt( 2*self.mu / (Cs133.mass * (self.wx**2)) )
        ry = np.sqrt( 2*self.mu / (Cs133.mass * (self.wy**2)) )
        rz = np.sqrt( 2*self.mu / (Cs133.mass * (self.wz**2)) )

        R_tf = np.array([rx, ry, rz])
        approxR_tf = np.array([round(rx, 10), round(ry, 10), round(rz, 10)])

        self.results["Thomas-Fermi Radius (m)"] = approxR_tf

        if printVal:
            return R_tf

    def xiCalc(self, printVal:bool):
        "function for computing the central healing length, returns the result in microns"
        xi = (8*np.pi*(self.mu/self.gInt)*self.aBr)**(-1/2)
        self.results["Central healing Length (microns)"] = round(xi*1e6, 5)

        if printVal:
            return xi*1e6

    def ctCalc(self, printVal:bool):
        "function for computing the critical temperature and the corrections, returns the result as an array with the critical temperature (nK), and the total correction as a fraction"
        self.TCrit = 0.94 * constants.hbar * self.w_ho * self.N**(1/3) / constants.k_B
        self.results["Critical Temperature (nK)"] = round(self.TCrit * 1e9, 5)

        #compute the corrections to the critical temperature
        dT_fsOverTCrit = -0.73 * (self.wAvg/self.w_ho) * self.N**(-1/3) #finite size correction
        dT_ihOverTCrit  = -1.33 * (self.aBr/self.a_ho) * self.N**(1/6) #inhomogenous correction
        TCor = dT_fsOverTCrit + dT_ihOverTCrit
        self.results["Critical Temperature Correction (%)"] = round(TCor, 5)

        if printVal:
            return np.array([self.TCrit*1e9, TCor])

    def aCritCalc(self, printVal:bool):
        "function for computing the critical scattering length, returns the result in meters"
        aCrit = 0.575 * self.a_ho / self.N / constants.a_0
        self.results["Critical Scattering Length (m)"] = round(aCrit, 5)

        if printVal:
            return aCrit

    def vCalc(self, printVal:bool):
        "function for computing the sound speed, returns the result in mm/s"
        vSound = np.sqrt(self.mu/Cs133.mass)
        self.results["sound speed (mm/s)"] = round(vSound *1e3, 5)

        if printVal:
            return vSound*1e3

    def lambdaCalc(self, printVal:bool):
        "function for computing the deBroglie wavelength, returns the result in microns"
        self.lambdaDB = np.sqrt( (2*np.pi*constants.hbar**2)/(Cs133.mass*self.mu) )
        self.results["deBroglie Wavelength (microns)"] = round(self.lambdaDB * 1e6, 5)

        if printVal:
            return self.lambdaDB * 1e6

    def phaseDensityCalc(self, printVal:bool):
        "function for computing the peak phase space density"
        phaseDensity = self.lambdaDB**3 * (self.mu/self.gInt)
        self.results["Phase space density"] = round(phaseDensity, 5)

        if printVal:
            return phaseDensity

    def genRes(self):
        "Generates results as a pandas data table"
        params = []
        vals = []

        #sort each parameter and value into lists
        for key in self.results:
            params.append(key)
            vals.append(self.results[key])

        #create the dataframe
        df = pd.DataFrame.from_dict({
            "parameter": params,
            "Values":vals
        })

        #display the dataframe
        display(df)
    
calc = BEC_calc(6.53,114,152,30e3,270,True)

,parameter,Values
0,Harmonic Oscillator Length (microns),1.25394
1,chemical Potential (nK),35.37183
2,Peak Density,53976595333311643648.0
3,Thomas-Fermi Radius (m),"[5.12739e-05, 2.937e-06, 2.2028e-06]"
4,Central healing Length (microns),0.22714
5,Critical Temperature (nK),67.79978
6,Critical Temperature Correction (%),-0.1286
7,Critical Scattering Length (m),0.45417
8,sound speed (mm/s),1.48756
9,deBroglie Wavelength (microns),0.80519


# Fermi Gas Chemical Potential